In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from PIL import Image
import numpy as np
import os

spark = SparkSession.builder.appName("ImageClassification").getOrCreate()


In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
from PIL import Image
import numpy as np
import os

# Spark oturumunu başlat
spark = SparkSession.builder.appName("ImageProcessing").getOrCreate()

# UDF oluştur
def process_image(image_path):
    try:
        img = Image.open(image_path)
        img = img.resize(size=(28, 28))
        img_data = np.array(img, dtype=np.float16) / 255
        return img_data
    except Exception as e:
        return None

process_image_udf = udf(process_image, StringType())

# Image dosyalarını yükle
image_df = spark.read.format("image").load("/Decks/Cracked")

# Dosya yolunda olmayan dosyaları filtrele
image_df = image_df.filter(image_df["image"].isNotNull())

# Image dosyalarını işlemek için UDF'yi kullan
processed_image_df = image_df.withColumn("processed_image", process_image_udf(image_df["image"]))

# Sınıflandırma işlemi için kullanılacak UDF
def udf_classify(img_data):
    if img_data is not None:
        # Sınıflandırma mantığını burada uygulayabilirsiniz
        return "Cracked"  # Varsayılan olarak "Cracked" olarak sınıflandırıyoruz
    else:
        return None

from pyspark.sql.functions import col

# UDF'yi DataFrame üzerinde kullanma
processed_image_df = processed_image_df.withColumn("class", udf_classify(col("processed_image")))

# Sonuçları görüntüle
processed_image_df.show()


TypeError: col should be Column

In [ ]:
if processed_image_df.count() > 0:
    first_row = processed_image_df.first()
    x = first_row["processed_image"]
    y = first_row["label"]
else:
    print("DataFrame is empty")


In [ ]:
# İlk satırı alalım
first_row = processed_image_df.first()

# processed_image ve label sütunlarının değerlerini alalım
x = first_row["processed_image"]
y = first_row["label"]

x, y = processed_image_df('/DATASET/Decks')